In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:

from pathlib import Path
from geovision.data.imagenette import Imagenette
imagenette_root = Path("../../../../datasets/imagenette").resolve()
Imagenette.transform_to_imagefolder(imagenette_root)

In [ ]:
from typing import Optional
import yaml
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision.transforms import v2 as T # type: ignore
#from torchvision.models import resnet18, ResNet18_Weights
from geovision.config.basemodels import ExperimentConfig # noqa
from geovision.data.module import ImageDatasetDataModule
#from geovision.training.module import ClassificationModule

transforms: dict[str, T.Transform | None] = {
    "image_transform": T.Compose([T.ToImage(), T.ToDtype(torch.float32)]),
    "target_transform": None,
    "common_transform": None,
}

config = ExperimentConfig.from_config_file("config.yaml", transforms)
datamodule = ImageDatasetDataModule(config)
#classifier = ClassificationModule(
    #config = config, 
    #model = resnet18(num_classes = config.dataset.num_classes)
#)

In [ ]:
datamodule.setup("fit")
image, target, idx = datamodule.train_dataset[0]

In [ ]:
from torch import float32
image.dtype == float32

In [64]:
import zipfile
import pandas as pd
import imageio.v3 as iio
from tqdm import tqdm
from pathlib import Path
from geovision.data.imagenet import Imagenet
from matplotlib import pyplot as plt

imagenet_root = Path("../../../../datasets/imagenet").resolve()
df = Imagenet.get_dataset_df_from_imagefolder(imagenet_root)
# Imagenet.transform_to_imagefolder(imagenet_root)
#image_paths = list(imagefolder.rglob("*.JPEG"))  + list(imagefolder.rglob("*.jpg"))
#imagefolder = (imagenet_root / "imagefolder" / "images")

In [3]:
# import shutil
# for image_path in tqdm(image_paths):
    # JPEG_path = image_path
    # jpg_path = image_path.parent / f"{image_path.stem}.jpg"
    # shutil.move(JPEG_path, jpg_path)

In [46]:
class_names = dict()
with open(imagefolder.parent/"LOC_synset_mapping.txt") as file:
    for line in file.readlines():
        label = line.split(' ')[0]
        name = '_'.join(line[10:].split(',')[0].lower().removesuffix('\n').split(' '))
        class_names[label] = name

In [59]:
class_labels = dict()
with open(imagefolder.parent/"LOC_synset_mapping.txt") as file:
    for line in file.readlines():
        class_labels[line.split(' ')[0]] = '_'.join(line[10:].split(',')[0].lower().removesuffix('\n').split(' '))
class_synsets = sorted(class_labels.keys())
class_names = tuple(class_labels[c] for c in class_synsets) 
df = (
    pd.DataFrame({"image_path": image_paths})
    .assign(image_path = lambda df: df["image_path"].apply(lambda x: Path(x.parents[1].stem, x.parents[0].stem, x.name)))
    .assign(label_str = lambda df: df["image_path"].apply(lambda x: x.parent.stem))
    .sort_values("label_str")
    .assign(label_str)
    #.assign(label_idx = lambda df: df["image_path"].apply(lambda x: class_synsets.index(x.parent.stem)))
    .reset_index(drop = True)
)
df

,image_path,label_idx,label_str
0,images/n01440764/n01440764_6786.jpg,0,tench
1,images/n01440764/n01440764_5642.jpg,0,tench
2,images/n01440764/n01440764_5638.jpg,0,tench
3,images/n01440764/n01440764_5636.jpg,0,tench
4,images/n01440764/n01440764_5634.jpg,0,tench
...,...,...,...
1331162,images/n15075141/n15075141_2192.JPEG,999,toilet_tissue
1331163,images/n15075141/n15075141_2186.JPEG,999,toilet_tissue
1331164,images/n15075141/n15075141_21835.JPEG,999,toilet_tissue
1331165,images/n15075141/n15075141_22391.JPEG,999,toilet_tissue


In [62]:
class_names

('tench',
 'goldfish',
 'great_white_shark',
 'tiger_shark',
 'hammerhead',
 'electric_ray',
 'stingray',
 'cock',
 'hen',
 'ostrich',
 'brambling',
 'goldfinch',
 'house_finch',
 'junco',
 'indigo_bunting',
 'robin',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'water_ouzel',
 'kite',
 'bald_eagle',
 'vulture',
 'great_grey_owl',
 'european_fire_salamander',
 'common_newt',
 'eft',
 'spotted_salamander',
 'axolotl',
 'bullfrog',
 'tree_frog',
 'tailed_frog',
 'loggerhead',
 'leatherback_turtle',
 'mud_turtle',
 'terrapin',
 'box_turtle',
 'banded_gecko',
 'common_iguana',
 'american_chameleon',
 'whiptail',
 'agama',
 'frilled_lizard',
 'alligator_lizard',
 'gila_monster',
 'green_lizard',
 'african_chameleon',
 'komodo_dragon',
 'african_crocodile',
 'american_alligator',
 'triceratops',
 'thunder_snake',
 'ringneck_snake',
 'hognose_snake',
 'green_snake',
 'king_snake',
 'garter_snake',
 'water_snake',
 'vine_snake',
 'night_snake',
 'boa_constrictor',
 'rock_python',
 'indian_cobr

In [ ]:
# Testing Dataset, DataLoader and Datamodule; What can go wrong?
# Dataset must have a specific interface -> DatasetABC
# Dataset must have a specific default_behaviour -> PyTest
# Dataset must verify root, split, df and transforms -> Validator
# Dataset must have a specific df schema -> Pandera.DataFrameSchema
#   1. Make sure all required columns are present and the right dtype
#   2. Make sure the sampling algorithms work as expected (no overlaps in tabular sampling, etc)
# Dataset must have a specific output shape and type -> PyTest 
#   1. Make sure dataset can load and output every single image in the df 
#   2. Make sure all dataset samples are the same shape 
# Dataset: plot_one_sample() and plot_batch()
# DataLoader: 

In [ ]:
import torch
conf1 = torch.nn.Conv2d(3, 1, 3, 3, 0, 1, 1, bias = True, padding_mode = "zeros")
conf1.bias
